# 1. Frame the problem
Using the customer description, Define the problem your trying to solve in your own words (remember this is not technial but must be specific so the customer understands the project

Using machine learning, identify handwritten cursive letters.

# 2. Get the Data 
Define how you recieved the data (provided, gathered..)

In [3]:
!pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 66.3 MB/s eta 0:00:0000:0100:01m

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2

In [56]:
import zipfile

with zipfile.ZipFile("cursiveData.zip", 'r') as zip_ref:
    zip_ref.extractall("Data2")

# 3. Explore the Data
Gain insights into the data you have from step 2, making sure to identify any bias

Some things I'm thinking about:
- black and white
- scale images
- canny edges

convert files

crop data so that each corner is touching white (should work?)

In [69]:
from pathlib import Path
import cv2
import glob

# look for types of file endings
png_count = len(glob.glob('Data2/Cursive/**/*.png', recursive=True))
jpg_count = len(glob.glob('Data2/Cursive/**/*.jpg', recursive=True))
jpeg_count = len(glob.glob('Data2/Cursive/**/*.jpeg', recursive=True))
heic_count = len(glob.glob('Data2/Cursive/**/*.HEIC', recursive=True))
print(png_count, jpg_count, jpeg_count, heic_count)

78 106 36 415


In [75]:
import os

# file folder counts
dirs = 0
files = 0

for root, dirnames, filenames in os.walk("Data2/Cursive"):
    dirs += len(dirnames)
    files += len(filenames)

print(files)

897


In [89]:
from collections import Counter

counts = Counter()

for f in Path("Data2/Cursive").rglob("*"):
        if f.is_file():
            counts[str(f.parent)] += 1
# print(counts)

for folder, num_files in sorted(counts.items()):
        if num_files != 26:
            print(f"{folder:} | {num_files}")

# get rid of these?

Data2/Cursive/S10/S13 | 76
Data2/Cursive/S11 | 2
Data2/Cursive/S14 | 5
Data2/Cursive/S17 | 25
Data2/Cursive/S33 | 25
Data2/Cursive/S34 | 10


## Conclusions

I need to:
- Label Data
- Crop images (content aware)
- Rotate (not sure)
- Black and white (opencv)
- Canny edge (highlight edges so the letters are easier to pick out)

# 4.Prepare the Data


Apply any data transformations and explain what and why


In [9]:
!pip install pyCAIR
!pip install natsort

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [81]:
destination_path = 'Data2/Cursive'
target_path = 'BetterData2'

format_of_your_images = 'png'

all_the_files = Path(destination_path).rglob(f'*.{format_of_your_images}')

for f in all_the_files:
    p = cv2.imread(str(f))
    gray = cv2.cvtColor(p, cv2.COLOR_BGR2GRAY)
    canny = cv2.Canny(gray, 50, 150)
    cv2.imwrite(f'{target_path}/{f.name}', canny)

In [40]:
image = cv2.imread("butt.jpg")
print(type(image), image.shape)

<class 'numpy.ndarray'> (2060, 1545, 3)


In [90]:
from pyCAIR.seam_carve import cropByColumn

seam_img, col_cropped = cropByColumn(image=image, display_seams=1, lsit=lsit)

  0%|          | 0/773 [00:00<?, ?it/s]


AttributeError: module 'numpy' has no attribute 'int'.
`np.int` was a deprecated alias for the builtin `int`. To avoid this error in existing code, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [ ]:
# scale images
# https://www.geeksforgeeks.org/python/resize-multiple-images-using-opencv-python/


# 5. Model the data
Using selected ML models, experment with your choices and describe your findings. Finish by selecting a Model to continue with

## Conclusions

# 6. Fine Tune the Model

With the select model descibe the steps taken to acheve the best rusults possiable 


# 7. Present
In a customer faceing Document provide summery of finding and detail approach taken


# 8. Launch the Model System
Define your production run code, This should be self susficent and require only your model pramaters 


In [56]:
# import libraries
import pandas as pd
import numpy as np
import joblib
from scipy.sparse import hstack, csr_matrix


def infrence(text):

    # load everything in
    model = joblib.load("./models/best_mnb.pkl")
    vectorizer = joblib.load("./models/vectorizer.pkl")
    scaler = joblib.load("./models/scaler.pkl")
    feature_params = joblib.load("./models/feature_params.pkl")
    
    s_words = feature_params[0]
    h_words = feature_params[1]
    good_emails = feature_params[2]

    # clean and tokenize the text
    cleaned = re.sub(r'[^A-Za-z\s]', '', text).lower()
    tokens = [word for word in cleaned.split() if word not in stopwords and len(word) > 2]
    token_counter = collections.Counter(tokens)

    # freq of most common words
    spam_freq = np.sum([token_counter[word.lower()] for word in s_words])
    ham_freq = np.sum([token_counter[word.lower()] for word in h_words])

    # amt of punctuation
    punctuation_chars = string.punctuation
    regex_pat = "[" + re.escape(punctuation_chars) + "]"
    punct_freq = len(re.findall(regex_pat, text))

    # good email domain
    good_email_count = sum(1 for x in good_emails if x in text.lower())

    # msg length
    msg_len = len(text)

    # create df of numeric
    numeric_features = pd.DataFrame([[spam_freq, punct_freq, msg_len, ham_freq, good_email_count]],
                                columns=['spam_most_common_freq','punctuation_freq','msg_len','ham_most_common_freq','good_email'])
    numeric_features_scaled = scaler.transform(numeric_features)

    # vectorize words
    text_features = vectorizer.transform([text])
    X = hstack([text_features, csr_matrix(numeric_features_scaled)])
    
    # predcit!
    prediction = model.predict(X)
        
    return prediction[0]

In [57]:
test1 = '''Subject: stock market information

sender : trinity venture , inc . address : 1911 westmead # 2710 , houston , tx 7707 call fax toll free : 877-393 - 7237 hours : 9 5 pm cst hi , receive e-mail address someone interest stock market information . e-mail informational purpose . offer sell solicit security . wish receive type information , please click " reply " put " remove " subject . permanently remove address our database . news help due diligence abrg , www . yahoo . com click " stock quote " . put abrg symbol lookup area click " quote " . happy invest ! - dean casia president trinity venture , inc . news release ambra resources group inc . # 610-800 west pender street vancouver , b . c . canada v6c 2v6 symbol abrg ( otc : bb ) acquisition venture oil & gas , inc . ambra resources group inc . adds projects ambra vancouver , british columbium - 4 , 1999 ambra resource group inc . ( otc bb : abrg ) , acquire 50 % ownership venture oil & gas , inc . ambra become 50 % owner property project venture interest . consideration ambra 's purchase 50 % capital stock venture inc . one million shares ambra common stock . part acquisition , venture assign ambra , 50 % interest bastian bay field prospect , state lease " 9800 " . 1 , plaquemine parish , louisiana . additional project assign ambra venture ambra shall right request assignment 50 % interest retain venture various project . primary business venture inc . acquire oil gas property re-mediation re-completion work result enhance recovery rate bring back commercial production . current economic condition petroleum industry facilitate acquisition property larger produce company declare surplus property . ambra venture inc . favorable position able acquire property while inventory offering high level . venture currently negotiate acquisition multiple oil gas project texa , louisiana oklahoma , party anticipate add significantly ambra 's resource base productive project . board director john m . hickey , president contact : ambra resource group inc . investor relation : 800-698 - 3377 604-669 - 2723 web site : http : / / www . ambraresource . com release informational purpose . offer sell solicit security product kind . release include information constitute forward-look statement pursuant safe harbor provision private security litigation reform act 1995 . forward-look statement involve risk uncertainty cause actual result differ materially future result encompass within forward-look statement . material provide ambra resource group inc . " " basis . ambra resource group inc . expressly disclaim warranty , express imply , include without limitation , warranty merchantability fitness particular purpose , respect service material product . event shall ambra resource group inc . liable direct , indirect , incidental , punitive consequential damages kind whatsoever respect service , material product . trinity venture , inc . receive fee $ 5 , 0 distribute document . permanently remove e-mail address our file call fax us toll free 877-393 - 7237 ; click " reply " put " remove " subject .
'''
pred = infrence(test1)
print("spam" if pred == 1 else "ham")

spam


In [58]:
test2 = '''Subject: postdoc position groningen , netherland

job position : postdoc dept . social pharmacy pharmacoepidemiology , groningen institute drug study , faculty mathematic natural science , netherland . description work group social pharmacy pharmacoepidemiology perform fundamental apply research epidemiological medical literature database order determine effectiveness / side-effect profile drug . post-doc ask participate program develop computer text analysis pattern recognition technique extraction ( side ) effect profile drug pharmaceutical medical electronic literature database : ( 1 ) source information lead innovative drug research ; ( 2 ) determine benefit-risk profile drug . phd - student assign program . requirement computer linguist computer scientist , complete phd - project expertise corpus linguistics , mathematical linguistics intelligent information retrieval ; interest pharmaceutical science innovative drug research ; expertise datum mine pattern recognition method desire . remark salary basis ministry guideline minimum dfl . 3844 , - maximum dfl . f . 7 . 125 , - ( schaal 10 / 11 rwoo ) bruto pro month , dependent education experience . work group social pharmacy pharmacoepidemiology part dutch school ' groningen - utrecht institute drug exploration ' ( guide ) , acknowledge royal dutch academy science . appointment two . information project : prof . dr . r . vo , email : r . vo @ farm . rug . nl ; tel . + 31 . 50 . 3633331 / 3633272 ; fax . + 31 . 50 . 3633311 . reaction . s . . p . , preferably before july 1st , 1997 . - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - marc weeber http : / / www . farm . rug . nl / marc / home . html groningen university centre pharmacy marc @ farm . rug . nl social pharmacy pharmacoepidemiology tel : + 31 50 3637571 _ _ _ . deusinglaan 2 fax : + 31 50 3633311 | 9713 aw groningen , netherland - - - - - - - - - - - - - - - - - - - - - 0 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
'''
pred = infrence(test2)
print("spam" if pred == 1 else "ham")

ham
